In [1]:
from datasets import load_dataset
import pinecone

dataset = load_dataset("squad")

Reusing dataset squad (C:\Users\danhi\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 200.05it/s]


In [2]:
dataset['train'][-1]

{'id': '5735d259012e2f140011a0a1',
 'title': 'Kathmandu',
 'context': "Kathmandu Metropolitan City (KMC), in order to promote international relations has established an International Relations Secretariat (IRC). KMC's first international relationship was established in 1975 with the city of Eugene, Oregon, United States. This activity has been further enhanced by establishing formal relationships with 8 other cities: Motsumoto City of Japan, Rochester of the USA, Yangon (formerly Rangoon) of Myanmar, Xi'an of the People's Republic of China, Minsk of Belarus, and Pyongyang of the Democratic Republic of Korea. KMC's constant endeavor is to enhance its interaction with SAARC countries, other International agencies and many other major cities of the world to achieve better urban management and developmental programs for Kathmandu.",
 'question': 'What is KMC an initialism of?',
 'answers': {'text': ['Kathmandu Metropolitan City'], 'answer_start': [0]}}

In [9]:
dataset_example = dataset['train'].map(
    lambda x :{'metadata':{'title':x['title']}}
)

Loading cached processed dataset at C:\Users\danhi\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-3bd0bf33e6994abd.arrow


In [11]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

dataset_example = dataset_example.map(
    lambda x: {
        'vector': embedder.encode(x['context']).tolist()
    }, batched=True, batch_size=16)

100%|██████████| 5475/5475 [07:17<00:00, 12.51ba/s]


In [12]:
dataset_example[0].keys()

dict_keys(['answers', 'context', 'id', 'metadata', 'question', 'title', 'vector'])

In [13]:
key = 'bed3cc56-6922-4ae9-ae26-775fb0197f69'
pinecone.init(api_key=key, environment='us-west1-gcp')

In [15]:
pinecone.create_index(name='squad-test', metric='euclidean', dimension=768)

In [16]:
index = pinecone.Index('squad-test')

In [19]:
from tqdm.auto import tqdm  # for progress bar

batch_size = 100
for i in tqdm(range(0, len(dataset_example), batch_size)):
    # set end of current batch
    i_end = i + batch_size
    if i_end > len(dataset_example):
        # correct if batch is beyond dataset size
        i_end = len(dataset-1)
    batch = dataset_example[i: i_end]
    # upsert the batch
    index.upsert(vectors=zip(batch['id'], batch['vector'], batch['metadata']))

In [18]:
index.describe_index_stats()

{'dimension': 768, 'namespaces': {'': {'vector_count': 87500}}}